# **Sampling Bayesian Regression**

Note that this was originally run on google colab so the spacing might need to be fixed.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import norm

import pymc3 as pm
import arviz as az

import pickle
import copy as copy

import itertools

In [ ]:
## Function reads data into a Pandas DataFrame from a CSV file ##

def read_data(filename):
    
    df = pd.read_csv(filename)
    
    # select required features
    d = df[['Season', 'Time of Day', 'Session', 'Driver Short Name', 'Full Lap']].copy()
    
    ''' some sectors have missing values (probably where car has been taken off, 
    convert the white space to NaN objects and drop them'''
    
    d.replace(' ', np.nan, inplace = True) 
                                              
    data = d.dropna()
    
    return data

In [ ]:
# read data
data_20 = read_data('07_05A1r_Qu_2020.csv')
data_19 = read_data('06_30A1r_Qu_2019.csv')
data_18 = read_data('07_01A1r_Qu_2018.csv')
data_17 = read_data('07_09A1r_Qu_2017.csv')

# split data into Q1 and Q2 per year
data_20_Q1 = data_20[data_20['Session'] == 'Q1']
data_20_Q2 = data_20[data_20['Session'] == 'Q2']

data_19_Q1 = data_19[data_19['Session'] == 'Q1']
data_19_Q2 = data_19[data_19['Session'] == 'Q2']

data_18_Q1 = data_18[data_18['Session'] == 'Q1']
data_18_Q2 = data_18[data_18['Session'] == 'Q2']

data_17_Q1 = data_17[data_17['Session'] == 'Q1']
data_17_Q2 = data_17[data_17['Session'] == 'Q2']

# order 2020 data by time (for real time updates)
data_20_Q1 = data_20_Q1.sort_values(['Time of Day'])
data_20_Q2 = data_20_Q2.sort_values(['Time of Day'])

# list of DataFrames in the historicals for Q1 and Q2
hist_data_Q1 = [data_19_Q1, data_18_Q1, data_17_Q1]
hist_data_Q2 = [data_19_Q2, data_18_Q2, data_17_Q2]

In [ ]:
## Find the exact start time of each session ##

timing = pd.read_csv('07_05A1r_TimeQu_2020.csv')
Q1_start_time = timing.iloc[0, 1]
Q2_start_time = timing.iloc[1, 1]

print(f'Q1 Start Time is: {Q1_start_time}, Q2 Start Time is: {Q2_start_time}')

Q1 Start Time is: 14:59:59.9230000, Q2 Start Time is: 15:24:59.8980000


In [ ]:
names_2020 = np.unique(data_20['Driver Short Name']) # finds the drivers that raced in 2020 Q1
names_2020_Q2 = np.unique(data_20_Q2['Driver Short Name']) # finds the drivers that raced in 2020 Q2

In [ ]:
## Fastest lap time per driver ##

def best_lap(data, name): # driver name is a string i.e 'HAM'

  if np.sum(data['Driver Short Name'] == name) > 0: # if there is data for the driver returns True
    
    df = data[data['Driver Short Name'] == name]
    min_idx = np.where(df['Full Lap'] == np.min(df['Full Lap']))[0]
    fastest = df.iloc[min_idx, :]

    return fastest
  
  else: # if there is not data for the driver
    return [0, 0, 0, 0, 0] # list of 5 - bigger than it can possible be so shows where there is missing data, 
                            # can use this to filter it out later


In [ ]:
## Store hisotircal best lap for all drivers for Q1 and Q2 2020 ##

lap_dict_Q1 = {}

for name in names_2020:
  laps = pd.DataFrame()

  for data in hist_data_Q1:
    lap = best_lap(data, name)

    if len(lap) < 5: # for drivers without historical data i.e LAT, just fill in blank lap (as found using best_lap function)
      laps = laps.append(lap)
  
  lap_dict_Q1[name] = laps


lap_dict_Q2 = {}

for name in names_2020_Q2:
  laps = pd.DataFrame()

  for data in hist_data_Q2:
    lap = best_lap(data, name)

    if len(lap) < 5:
      laps = laps.append(lap)
  
  lap_dict_Q2[name] = laps

In [ ]:
lap_dict_Q2.keys()


dict_keys(['ALB', 'BOT', 'GAS', 'GRO', 'HAM', 'KVY', 'LEC', 'NOR', 'OCO', 'PER', 'RIC', 'SAI', 'STR', 'VER', 'VET'])

In [ ]:
## Model 1 to get the posterior on parameters given data = prior for model 2 ##

def fitBR(inputs, observed, initial_params = None): 

  if initial_params != None:
    initial_slope, inital_sigma, initial_intercept = initial_params # initial params should be a list of tuples (mu, std)
  
  else: # set non-informative priors for the parameters that have no historical data
    initial_slope = (0, 10) 
    initial_intercept = (0, 10)
    initial_sigma = (10,)

  with pm.Model() as model:
    intercept = pm.Normal('Intercept', mu = initial_intercept[0], sd = initial_intercept[1])
    slope = pm.Normal('slope', mu = initial_slope[0], sd = initial_slope[1])
    sigma = pm.HalfNormal('sigma', sd = initial_sigma[0])
    #sigma = 1 

    mean = intercept + slope * inputs
    likelihood = pm.Normal('Y_obs', mu = mean, sd = sigma, observed = observed)
    step = pm.NUTS() # run with gibbs
    posterior = pm.sample(3000, step)

  return posterior


In [ ]:
## Function modified from: https://docs.pymc.io/pymc-examples/examples/pymc3_howto/updating_priors.html ##

''' feed in trace for each parameter (intercept, slope, sigma) and the function fits a gaussian based on those samples and generates points between min sample
and max sample to get a distribution '''

def from_posterior(param, samples): 

    smin, smax = np.min(samples), np.max(samples) # samples from posterior (trace), takes the mean and max and fit another gaussian from posterior distribution
    width = smax - smin
    x = np.linspace(smin, smax, 100) 
    y = sp.stats.gaussian_kde(samples)(x) # gaussian evaluated at points where samples are, y is probability of samples

    if param == 'sigma': # standard deviation cannot be negative, throws error if this is not changed
      x = np.concatenate([x, [x[-1] + 3 * width]])
      y = np.concatenate([y, [0]])
    
    else:
      # what was never sampled should have a small probability but not 0,
      # so extend the domain and use linear approximation of density on it
      x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]]) # add points that are 3 ranges away from both ends and assume probs are 0, so it knows when to stop interpolating
      y = np.concatenate([[0], y, [0]])

    return pm.distributions.Interpolated(param, x, y) 

In [ ]:
## Model 2 to update the posterior as new data gets fed in ##

def updateBR(inputs, lap_time, posterior): 

  with pm.Model() as model2:

    # set priors
    model_slope = from_posterior('slope', posterior['slope'])
    model_intercept = from_posterior('Intercept', posterior['Intercept'])
    model_sigma = from_posterior('sigma', posterior['sigma'])

    mean =  model_slope * inputs + model_intercept

    likelihood = pm.Normal('Y_obs', mu = mean, sd = model_sigma, observed = lap_time)

    step = pm.NUTS()

    posterior2 = pm.sample(1000, step)

  return posterior2

In [ ]:
## Function returns what the predicted best laps would would be for each sample: y = mx + c
## it returns the mean and standard deviations of these predictions ##

def predict(input, traces):

  preds = traces['slope'] * input + traces['Intercept']
  mean_pred = np.mean(preds)
  std_pred = np.std(preds)

  return mean_pred, std_pred

In [ ]:
## Find the priors for all drivers using Model 1 and stores them ##

historical_data = lap_dict_Q1 # change to lap_dict_Q2 for Q2

driver_prior = {}

for driver in names_2020: # change to names_2020_Q2 for Q2

  print(f'Current Driver: {driver}')
  driver_data = historical_data[driver]

  if len(driver_data) > 0:
    driver_data = driver_data.loc[:, ['Season', 'Full Lap']]
    inputs = np.array(driver_data['Season'].astype('float64')) # feature - the season
    observed = np.array(driver_data['Full Lap']) # label - full lap time (which is best lap per year from pre-processing above)

    driver_posterior = fitBR(inputs, observed) # fit historicals using Model 1

    driver_prior[driver] = driver_posterior # update prior dictionary

  else: # if there is no historical data for a driver then have an empty list for the prior
    driver_prior[driver] = []

Current Driver: ALB


KeyboardInterrupt: ignored

In [ ]:
## Function will save the prior dictionary created as a pickle file so that Model 1 does not have to be run again ##

def savePickle(prior, name):

    with open(name + '.pickle', 'wb') as handle:
        pickle.dump(prior, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
## Function will open pickle file ##

def openPickle(name):
  
    filename = name + '.pickle'

    with open(filename, 'rb') as file:
        prior = pickle.load(file)

    return prior

In [ ]:
savePickle(driver_prior, 'driver_prior_A1r_Q2')

In [ ]:
driver_prior = openPickle('driver_prior_A1r_Q1')

In [ ]:
## Store driver names as an index and keep dictionary both ways to later identify drivers ##

name_to_idx = {} 
idx_to_name = {}

idx = 0

for name in names_2020: # change to names_2020_Q2 for Q2
  name_to_idx[name] = idx
  idx_to_name[idx] = name
  idx += 1

In [ ]:
name_to_idx

{'ALB': 0,
 'BOT': 1,
 'GAS': 2,
 'GIO': 3,
 'GRO': 4,
 'HAM': 5,
 'KVY': 6,
 'LAT': 7,
 'LEC': 8,
 'MAG': 9,
 'NOR': 10,
 'OCO': 11,
 'PER': 12,
 'RAI': 13,
 'RIC': 14,
 'RUS': 15,
 'SAI': 16,
 'STR': 17,
 'VER': 18,
 'VET': 19}

In [ ]:
## Initialize everything - set first predictions per driver to lap time calculated from Model 1 ##

current_predictions = []
current_pred_std = []

input = 2020

for name in names_2020: # change to names_2020_Q2 for Q2
  if len(driver_prior[name]) > 0:
    mean_pred, std_pred = predict(input, driver_prior[name])
    current_predictions.append(mean_pred)
    current_pred_std.append(std_pred)

  else:
    current_predictions.append(np.inf) 
    current_pred_std.append(np.inf)

current_fastest = np.array([np.inf] * len(names_2020)) # change to names_2020_Q2 for Q2

i = np.argsort(current_predictions)[-5] # 16th position for Q1, 11th position for Q2 - slowest 5 will not make it through in both rounds

print(f'Initial mean predictions from historical data are: {current_predictions}')
print(f'Initial standard deviation predictions from historical data are: {current_pred_std}')
print(f'Initial predicted cut-off is: {current_predictions[i]}')

Initial mean predictions from historical data are: [65.04439344516635, 64.71692287918117, 64.53455143788399, 64.33382079793763, 64.98904351522783, 64.18578177639417, 65.65401193002101, inf, 64.43864027024173, 65.07093773998767, 64.3941049338923, 65.5084425059258, 65.33395099062828, 64.6144959622954, 65.09734562546144, 65.80208692743088, 64.77125716576221, 65.58117006527992, 64.70201087681512, 64.61482086999676]
Initial standard deviation predictions from historical data are: [9.922051039481234, 2.550176714549966, 4.953148484801849, 10.61475879011319, 2.423257499348498, 1.9279557449607314, 4.098186498114872, inf, 4.238460961192324, 2.0075983505151993, 10.577659724900482, 4.362993192080829, 2.18885999706339, 2.5024651857522873, 2.2344625100853723, 9.570952635291476, 1.6867146007500138, 2.0632250564658117, 2.3451717076503833, 2.2386955833507844]
Initial predicted cut-off is: 65.5084425059258


In [ ]:
# If any of the drivers did not have prior historical information and initial prediction got set to np.inf then we will replace it with the mean of the other predictions (this is so that the probability function will work))

current_predictions = np.array(current_predictions)
current_pred_std = np.array(current_pred_std)
mean_of_means = np.mean(current_predictions[current_predictions != np.inf])
mean_of_sd = np.mean(current_pred_std[current_pred_std != np.inf])

current_predictions[current_predictions == np.inf] = mean_of_means
current_pred_std[current_pred_std == np.inf] = mean_of_sd

In [ ]:
## Store all combinations of driver indexes for 19C4, 19C3, 19C2, 19C1, 19C0 (for Q2 this would be: 14C4, 14C3, 14C2, 14C1, 14C0) ##

combinations_list = []
for i in range(5):
  combinations = list(itertools.combinations(np.arange(19), i)) # change from 19 to 14 in Q2
  combinations_list.append(combinations)

In [ ]:
## Finds a list of the indexes of the drivers that are not the driver we are calculating the probability for ##

def idx_list(target_name):
  
  names_2020_idx = []

  for name in names_2020: # change to names_2020_Q2 for Q2

    if name != target_name: # to find the indexes of the other drivers not the one we are calculating the probability for

      value = name_to_idx[name]
      names_2020_idx.append(value)

  return names_2020_idx


In [ ]:
## Calculate probabilities of drivers being slower than lap set for driver in question, P(driver > lap time) ##

# z transform of params first (think scipy does this automatically), find cdf area under curve to the right of the mean

def calculate_prob(target, mean, sd, max = 100, min = 0, plot = False):

  cdf = norm.sf(x = target, loc = mean, scale = sd) # sf = survival function returns 1 - cdf

  if plot == True: # option to plot the pdf curve with the cdf area highlighted
    
    fig, ax = plt.subplots()
    x_axis = np.arange(min, max, 0.001)
    ax.plot(x_axis, norm.pdf(x, loc = mean, scale = sd))
    ax.set_title(f'$N \sim ({mean}, ${sd}^2$)$')
    ax.set_xlabel('x')
    ax.set_ylabel('pdf(x)')
    ax.grid(True)
    px = np.arange(target, max, 0.01)
    ax.set_ylim(0, 0.25)
    ax.fill_between(px, norm.pdf(px, loc = 3, scale = 2), alpha = 0.5, color = 'g')
    ax.text(4.5, 0.02, f'cdf = {cdf}', fontsize = 15)
    plt.show()

  else:

    pass

  return cdf

In [ ]:
## Function works out the probability of the specified driver being safe when they set a specified target lap ##

''' look at the report for how this works and is derived - I have tried to be thorough in the comments here - CSB Aug 2021 '''

def driver_safe(target_name, target_lap):
  
  # assume that current_predictions and combinations_list are both global as they will update with the BR model
  
  names_2020_idx = idx_list(target_name) # find indexes of the drivers who are not the driver that we want to calculate the probability for

  all_probs = np.array([calculate_prob(target_lap, current_predictions[idx], current_pred_std[idx]) for idx in names_2020_idx]) # calculate the probabilty of all other drivers being slower than the lap set for the target driver
  all_probs = 1 - (all_probs) #  1 - p(driver < target_lap) = p(driver > target lap), store all drivers like this and index the ones that should be slower to speed the code up (see prob_safe function)

  log_unsafe = prob_unsafe(all_probs, combinations_list) # log probability of target driver being unsafe

  return 1 - np.exp(log_unsafe) # probability of target driver being safe

# calculates the probability
def prob_unsafe(all_probs, combinations_list):

  log_comb = []

  for combinations in combinations_list: # i.e for i in [19C4, 19C3, ....]

    log_comb_probs = [] 

    for combination in combinations: # i.e for i in [(1, 2, 3, 4), (4, 3, 18, 12), ..., (0, 3, 9, 11)]

      combination = list(combination) # convert the combination tuple into a list so that it can be used to index below
      probs = copy.deepcopy(all_probs) # create a copy to be safe
      probs[combination] = 1 - probs[combination] # p(driver > target lap) for those in the combination list i.e those that are slower than the target lap and witll be in lower positions than the target driver
      probs = probs + 1e-30 * (probs < 1e-30) # add on a tiny negligable number to any probabilities that are 0 so they become slightly larger, this stops it crashing when we take log(0)
      log_probs = np.log(probs) # take the log of each probability calculated for that particular combination
      log_comb_probs.append(np.sum(log_probs)) # sum the log probabilities (same as multiplying the non-log probabilities) and append each to the list above

    log_comb_probs = np.array(log_comb_probs) # convert the log_probs to an array so we can use the elementwise operations max and sum
    factor = np.max(log_comb_probs) # before we take the exponent of the log probs its benefical to pull out the largest value as a factor so that we can have the exponent of a reasonably sized number, it gets added on at the end
    log_all_comb_probs = log_comb_probs - factor # take away the factor from all elements of this array 
    log_all_comb_prob = np.log(np.sum(np.exp(log_all_comb_probs))) + factor # take the exponent of log probabilities for each combination, add them and take the log, then add on the factor 

    log_comb.append(log_all_comb_prob) # this is the total log probability for 19CX combinations

  # sum all combinations i.e 19C4 + 19C3 + 19C2 + 19C1 + 19C0, need to use the factor trick again
  log_comb = np.array(log_comb)
  factor = np.max(log_comb)
  log_all_comb = log_comb - factor
  log_all_comb = np.log(np.sum(np.exp(log_all_comb))) + factor

  return log_all_comb # log probability of target driver being unsafe



In [ ]:
safe = driver_safe('RUS', 66)
safe

0.8662275285792096

In [ ]:
# update all drivers probabilities of being safe in one go

def update_safe_probs(probability_of_being_safe, observed):

  for idx in range(20): # change to 15 for Q2
    probability_of_being_safe[idx] = driver_safe(target_name = idx_to_name[idx], target_lap = observed)

  return probability_of_being_safe

In [ ]:
## Update for drivers sequentially and predict cutoff at each update ##

Q_data = data_20_Q1 # change to data_20_Q2 for Q2
cutoff_times = []
times = []
predictions = {}
probability_of_being_safe = [0] * 20 # set original probabilities of being safe to 0 and store current calculation

for idx in range(len(Q_data)): # for each new lap recorded

  lap = Q_data.iloc[idx, :]
  input = lap['Season']
  time = lap['Time of Day']
  driver = lap['Driver Short Name']
  observed = lap['Full Lap']
  print(driver)
  driver_idx = name_to_idx[driver] # convert driver name to driver index

  if current_fastest[driver_idx] > observed:
    current_fastest[driver_idx] = observed # update fastest lap for the driver in question and update model

    # if prior exists, then update weights using informative prior
    if len(driver_prior[driver]) > 0:
      prior = driver_prior[driver]
      driver_prior[driver] = updateBR(input, observed, prior)

    else:
      driver_prior[driver] = fitBR(input ,observed) # use non-informative prior

    # make new prediction for that driver (0 index for mean, 1 for standard deviation)
    prediction = predict(input, driver_prior[driver])
    current_predictions[driver_idx] = prediction[0]
    current_pred_std[driver_idx] = prediction[1]

    #probability_of_being_safe[driver_idx] = driver_safe(target_name = idx_to_name[driver_idx], target_lap = observed) # calculate the probability of the driver which has updated being safe - alternatively run this for all drivers at every update so it continually updates
    probability_of_being_safe = update_safe_probs(probability_of_being_safe, observed)

    # store all of the predictions at that time stamp - might be useful for plots later
    predictions[time] = current_predictions, current_pred_std

  times.append(time)
  cutoff_idx = np.argsort(current_predictions)[-5] # 16th position for Q1, 11th position for Q2 - slowest 5 will not make it through in both rounds
  cutoff_times.append(current_predictions[cutoff_idx])
  print(probability_of_being_safe)


LAT


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, slope, Intercept]


Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 12 seconds.
There were 246 divergences after tuning. Increase `target_accept` or reparameterize.
There were 764 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6773713694563003, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


[0.585443875913414, 0.6345144992311567, 0.6094855257139651, 0.5900439372583203, 0.6309407119400374, 0.6558772317158408, 0.5962334668890189, 0.5690535496629634, 0.6170462934427277, 0.6377520679002607, 0.5896379284419035, 0.5973452065406728, 0.6269011547305152, 0.6374600099771537, 0.6321114921605882, 0.5792189784646162, 0.6519740565416061, 0.6223025691131716, 0.6387309576196434, 0.6426130536267272]
GIO


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, Intercept, slope]


...

Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 8 seconds.
There were 79 divergences after tuning. Increase `target_accept` or reparameterize.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


[0.9197992986647444, 0.9253306147636964, 0.9219974028001799, 0.9148093416195466, 0.9185391503007945, 0.925611795439177, 0.9178218017841536, 0.9129387472463034, 0.9242836765265322, 0.9209062878495303, 0.9174204138542678, 0.9184194075303589, 0.9257965293249637, 0.9173269419493146, 0.9255090759811218, 0.9172781472043603, 0.9322241097733277, 0.92711684103922, 0.9260098431385491, 0.925622594078958]
HAM


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, Intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 9 seconds.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
There were 257 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.29543812769381256, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


[0.99773602151958, 0.9978495267156604, 0.9977537460808461, 0.9975351138868772, 0.9973721520080278, 0.9977024112047759, 0.9975092894100562, 0.9974512215624883, 0.997712846142292, 0.9972109672795991, 0.997694985756877, 0.997568391313584, 0.9976886323550355, 0.9974656301343782, 0.9976248477023331, 0.9976010466699848, 0.9976719616417109, 0.9977783938194971, 0.9978682595684697, 0.9977244962102967]


In [ ]:
print(cutoff_times) # Q2

[65.07061581001201, 65.07061581001201, 64.83526408011979, 64.8029711701992, 64.8029711701992, 64.8029711701992, 64.8029711701992, 64.8029711701992, 64.83526408011979, 64.83526408011979, 64.83526408011979, 64.8029711701992, 64.6780676025523, 64.58627801399855, 64.58627801399855, 64.66805230320422, 64.66805230320422, 64.66805230320422, 64.55309154683148, 64.55309154683148, 64.55309154683148, 64.55309154683148, 64.52164234478495, 64.52164234478495, 64.52164234478495, 64.52164234478495, 64.52164234478495, 64.52164234478495, 64.52164234478495, 64.13909853488971, 64.13374780056988, 64.13374780056988]


In [ ]:
print(cutoff_times) # Q1

[65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.33395099062828, 65.33395099062828, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.3975762736211, 65.26475016139415, 65.26475016139415, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.25370602553224, 65.2343097046226, 65.234309704622

In [ ]:
print(cutoff_times) # Q1

[65.5084425059258, 65.5084425059258, 65.58117006527992, 65.58117006527992, 65.58117006527992, 65.58117006527992, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.33395099062828, 65.33395099062828, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.5084425059258, 65.51213581400748, 65.5084425059258, 65.5084425059258, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.3399616386779, 65.27033241331931, 65.27033241331931, 

In [ ]:
print(times[-19])

15:15:26.4770000


In [ ]:
print(cutoff_times[-19])

65.27033241331931


In [ ]:
print(probability_of_being_safe)

[0.9320904331130021, 0.9384749700763638, 0.9287916257238303, 0.9199888190263273, 0.9390545879891183, 0.9271866418307435, 0.9289995431694759, 0.9309366221361629, 0.926761863316428, 0.9305349309520958, 0.9285028364146817, 0.9300265379843645, 0.9307574425306593, 0.9281440218062061, 0.9283400103633535, 0, 0, 0, 0, 0]


In [ ]:
print(probability_of_being_safe[7])

0.9045274335351178


In [ ]:
name_to_idx

{'ALB': 0,
 'BOT': 1,
 'GAS': 2,
 'GIO': 3,
 'GRO': 4,
 'HAM': 5,
 'KVY': 6,
 'LAT': 7,
 'LEC': 8,
 'MAG': 9,
 'NOR': 10,
 'OCO': 11,
 'PER': 12,
 'RAI': 13,
 'RIC': 14,
 'RUS': 15,
 'SAI': 16,
 'STR': 17,
 'VER': 18,
 'VET': 19}